In [ ]:
#imports

import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.metrics import precision_score
import nltk
import random
nltk.download('stopwords')

from nltk.corpus import stopwords
import string
stopset = stopwords.words('english') + list(string.punctuation)
nltk.download('punkt')
import re



import io


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!sudo apt-get install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [ ]:
#load train answer/clue embeddings

drive.mount('/content/drive', force_remount=True)

clue_embed_df = pd.read_csv('/content/drive/MyDrive/berttrainclue.csv')
answer_embed_df = pd.read_csv('/content/drive/MyDrive/berttrainanswer.csv')
clue_embed = np.array(clue_embed_df, order = 'c', dtype='float32')
answer_embed = np.array(answer_embed_df, order = 'c', dtype='float32')

Mounted at /content/drive


In [ ]:
#set up FAISS L2 index with 50 voronoi cell split, retrieve 1000 most similar answer embeddings to each clue

import faiss
nlist = 50
quantizer = faiss.IndexFlatL2(256)
index = faiss.IndexIVFFlat(quantizer, 256, nlist)
index.train(answer_embed)
index.add(answer_embed)
D, I = index.search(clue_embed, 1000)

In [ ]:
#train recall

np.mean([bool(i in I[i]) for i in range(len(I))])

0.4608269723573525

In [ ]:
del clue_embed

In [ ]:
del index, quantizer

In [ ]:
del D

In [ ]:
#export I matrix

pd.DataFrame(I).to_csv('/content/drive/MyDrive/bertI_train.csv', index=False)

In [ ]:
del I

In [ ]:
#load validation clue embeddings

clue_embed_df = pd.read_csv('/content/drive/MyDrive/bertvalclue.csv')
clue_embed = np.array(clue_embed_df, order = 'c', dtype='float32')

In [ ]:
#FAISS process for validation set

nlist = 50
quantizer = faiss.IndexFlatL2(256)
index = faiss.IndexIVFFlat(quantizer, 256, nlist)
index.train(answer_embed)
index.add(answer_embed)
D, I = index.search(clue_embed, 1000)

In [ ]:
#load train/val/test sets for answers

nyt_train = pd.read_csv('/content/drive/MyDrive/nyttrain.csv')
nyt_val = pd.read_csv('/content/drive/MyDrive/nytval.csv')
nyt_test = pd.read_csv('/content/drive/MyDrive/nyttest.csv')

In [ ]:
#answer sets

train_answers = np.array([k for j in [list(eval(i)) for i in nyt_train['answer']] for k in j])
val_answers = np.array([k for j in [list(eval(i)) for i in nyt_val['answer']] for k in j])
test_answers = np.array([k for j in [list(eval(i)) for i in nyt_test['answer']] for k in j])

In [ ]:
np.mean([bool(val_answers[i] in train_answers[I[i]]) for i in range(len(val_answers))])

0.3820897601058459

In [ ]:
pd.DataFrame(I).to_csv('/content/drive/MyDrive/bertI_val.csv', index=False)

In [ ]:
del clue_embed, index, quantizer, D, I

In [ ]:
#test clue embeddings

clue_embed_df = pd.read_csv('/content/drive/MyDrive/berttestclue.csv')
clue_embed = np.array(clue_embed_df, order = 'c', dtype='float32')

In [ ]:
#FAISS process for test set

nlist = 50  
quantizer = faiss.IndexFlatL2(256)
index = faiss.IndexIVFFlat(quantizer, 256, nlist)
index.train(answer_embed)
index.add(answer_embed)
D, I = index.search(clue_embed, 1000)

In [ ]:
#test recall

np.mean([bool(test_answers[i] in train_answers[I[i]]) for i in range(len(test_answers))])

0.3746765847347995

In [ ]:
#export I matrix

pd.DataFrame(I).to_csv('/content/drive/MyDrive/bertI_test.csv', index=False)

In [ ]:
del clue_embed, index, quantizer, D, I